<a href="https://colab.research.google.com/github/jkhyjkhy/NLP_Project/blob/main/Summarize_in_3_sentences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import requests
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import math
import numpy as np
from sklearn.preprocessing import normalize
from _datetime import datetime


In [10]:
def get_soup(url): # soup 객체를 가져옴
    res = requests.get(url)
    if res.status_code == 200:
        return bs(res.text, 'html.parser')
    else:
        print(f"Super big fail! with {res.status_code}")


In [11]:
def get_head(soup):
    head = soup.find('div', attrs={"class": "news_headline"})
    date_info = head.find('div', attrs={"class": "info"})

    print("기사 제목 : " + head.select_one('.title').get_text())
    print("기사 작성 시간 : " + date_info.select_one('span').get_text())
    # print("기자 이름 : " + head.find('em', attrs ={"class" : "media_end_head_journalist_name"}).get_text()[0:3]) , 적용이 안되는 기자가 많아 예외 처리


In [12]:
def get_content(soup):
    article = soup.find('div', attrs={"id": "newsEndContents"})

    content = article.text.replace("\n","")
    content = re.split("[\.?!]\s+", content) # 문장을 요소 단위로 분화, 문장 구분


    return content

In [13]:
def get_list(url): # 이 시각 많이 본 뉴스에서 리스트를 가져옴
    print(f"현재 시간은 {datetime.now()}입니다. 상위 10개의 기사를 조회합니다.")
    soup = get_soup(url)
    # news_list = soup.find('ol', attrs={"class" : "news_list"})
    news_list = soup.select('.news_list') # select를 써 본 기억이 없는것 같아 select로 구현
    news_link = {} # 딕셔너리로 선언
    for li in range(0, 10):
        print(f"{li+1}번 기사 : {news_list[0].findAll('a')[li].get_text()}")
        news_link[li] = news_list[0].findAll('a')[li].get('href')

    print("================================")

    checking_number = int(input("조회하고 싶은 기사를 입력하세요 : ")) # 조회를 원하는 기사 본문의 url 저장

    return "https://sports.news.naver.com/" + news_link[checking_number-1]

In [14]:
def textrank(x, df=0.85, max_iter=50): # df = Dumping Factor : 0.85라고 가정, max_iter = 최대 반복횟수, 제한 조건을 두고 적절한 값에 수렴할떄까지 반복해야 하나 임의적으로 50회라고 지정
    assert 0 < df < 1

    # initialize
    A = normalize(x, axis=0, norm='l1')
    R = np.ones(A.shape[0]).reshape(-1, 1)
    bias = (1 - df) * np.ones(A.shape[0]).reshape(-1, 1)
    # iteration
    for _ in range(max_iter):
        R = df * (A * R) + bias # @ 연산으로 한번에 계산해도 됨

    return R

In [15]:
def summarize_text(content):
    data = []
    for text in content:
        if (text == "" or len(text) == 0):
            continue
        elif text == "All right reserved": # 기사가 끝났으면 반복문 종료
            break
        temp_dict = dict()
        temp_dict['sentence'] = text
        temp_dict['token_list'] = text.split()  # 기초적인 띄어쓰기 단위로 나누기
        # 한국어 전처리 분석기인 꼬꼬마 분석기 사용도 고려해봐야 할 것 같음
        data.append(temp_dict)

    data_frame = pd.DataFrame(data)
    # print(data_frame) # 문장 리스트와 토큰화 된걸 보려면 주석 해제할 것
    print("================================")

    # 여기서부터
    # reference : https://hoonzi-text.tistory.com/68, 문서 요약 하기 (with textrank)
    # reference2 : https://lovit.github.io/nlp/2019/04/30/textrank/, TextRank 를 이용한 키워드 추출과 핵심 문장 추출 (구현과 실험)

    similarity_matrix = []
    for i, row_i in data_frame.iterrows():
        i_row_vec = []
        for j, row_j in data_frame.iterrows():
            if i == j:
                i_row_vec.append(0.0)
            else:
                intersection = len(set(row_i['token_list']) & set(row_j['token_list'])) # 유사도 계산의 분자 부분
                log_i = math.log(len(set(row_i['token_list'])))
                log_j = math.log(len(set(row_j['token_list'])))
                similarity = intersection / (log_i + log_j)
                i_row_vec.append(similarity)

        similarity_matrix.append(i_row_vec)

        weightedGraph = np.array(similarity_matrix)

    R = textrank(weightedGraph)
    R = R.sum(axis=1)

    indexs = R.argsort()[-3:] # 랭크값 상위 세 문장의 인덱스를 가져옴
    for index in sorted(indexs): # 뉴스 구조의 순서를 유지하기 위해 등장한 순서대로 정렬함
        print(data_frame['sentence'][index])
        print()


In [16]:
if __name__ == '__main__':
    url = get_list("https://sports.news.naver.com/wfootball/index")
    soup = get_soup(url)
    get_head(soup)
    print("================================")
    get_content(soup)
    summarize_text(get_content(soup))

현재 시간은 2023-11-05 15:03:49.260696입니다. 상위 10개의 기사를 조회합니다.
1번 기사 : 재계약 거부하고 이적 결심...토트넘 보강 계획 ‘청신호’ 잡혀
2번 기사 : "SON, 골 좀 그만 넣어"…아스널 DF, 퀴즈쇼에서 '토트넘 캡틴' 손흥민 극찬
3번 기사 : ‘이강인 멀티포지션 능력 덕분에 뎀벨레까지 살아났다!’ 엔리케 감독 함박웃음 지은 이유
4번 기사 : '내 뒤에 부폰 있다!'…'요리스 삭제해 버린' 토트넘 신입 GK, 다 이유가 있었네! "부폰이 항상 가치 있는 조언 해줘"
5번 기사 : "이강인 처음봤을 때 놀랐다" 700억 수비수 극찬…프랑스 매체들도 호평
6번 기사 : 이대로 가면 51골! '美친 골감각' 케인, 레반도프스키 41골 기록 넘어 '역대 최다골 득점왕' 가능성 ↑
7번 기사 : '포버지'가 밝힌 '매디슨X비카리오X판 더 펜 영입 성공 비결'..."선수가 아닌 사람으로 본다"
8번 기사 : 손흥민 어떻게 잊나…"포체티노 토트넘 복귀 원했다"
9번 기사 : 첼시 20살 DF의 도발 "토트넘이 우릴 언제 이겼더라? 기억 안 나"
10번 기사 : 무대 옮겨도 잘한다...케인, '첫 10경기 15골' 분데스 역사상 최다 득점 신기록+첫 데어클라시커 해트트릭 주인공
조회하고 싶은 기사를 입력하세요 : 2
기사 제목 : "SON, 골 좀 그만 넣어"…아스널 DF, 퀴즈쇼에서 '토트넘 캡틴' 손흥민 극찬
기사 작성 시간 : 기사입력 2023.11.05. 오후 10:01
(엑스포츠뉴스 권동환 기자) 아스널 수비수 윌리엄 살리바가 라이벌 클럽 주장 손흥민(토트넘 홋스퍼) 사진을 단번에 맞추면서 칭찬을 아끼지 않았다.영국 '스카이스포츠'는 지난 4일(한국시간) 공식 유튜브 채널을 통해 "윌리엄 살리바가 AI가 만든 프리미어리그 아기들을 추측했다"라며 한 영상을 게시했다.2001년생 프랑스 센터백 살리바는 프리미어리그 명문 아스널 핵심 수비수 중 한 명이다

당장 살리바는 지난 9월 2023/24시즌 프리미어리그 6라운드 